# Imports JSON, travail sur les lemmes et typologie des variantes

In [100]:
# On importe plusieurs bibliothèques, dont celle de pie (pour l'annotations linguistique) et celle de collatex.
import collatex
import time
import json
import pie
import subprocess

Commençons par lemmatiser notre texte

In [93]:
def lemmatize(path):
    device = "cpu"
    batch_size = 500
    modele = "model.tar"
    cmd = f'pie tag --device {device} --batch_size {batch_size} {path} <{modele},lemma,pos,Person,Numb,Tense,Case,Mood>'
    subprocess.run(cmd.split())
    print(f"Texte annoté enregistré sous {path.replace('.txt', '-pie.txt')}")

In [94]:
lemmatize("Catullus/TEXT-Bodmer47-1.txt")
lemmatize("Catullus/TEXT-O1.txt")
lemmatize("Catullus/TEXT-G2.txt")

 - model: model.tar
 - tasks: lemma, pos, Person, Numb, Tense, Case, Mood
Tagging file [Catullus/TEXT-Bodmer47-1.txt]...


100%|██████████| 11/11 [00:00<00:00, 142.51it/s]


Texte annoté enregistré sous Catullus/TEXT-Bodmer47-1-pie.txt
 - model: model.tar
 - tasks: lemma, pos, Person, Numb, Tense, Case, Mood
Tagging file [Catullus/TEXT-O1.txt]...


100%|██████████| 10/10 [00:00<00:00, 149.37it/s]


Texte annoté enregistré sous Catullus/TEXT-O1-pie.txt
 - model: model.tar
 - tasks: lemma, pos, Person, Numb, Tense, Case, Mood
Tagging file [Catullus/TEXT-G2.txt]...


100%|██████████| 11/11 [00:00<00:00, 145.94it/s]


Texte annoté enregistré sous Catullus/TEXT-G2-pie.txt


In [96]:
# On vérifie que l'annotation a fonctionné sur un des textes:
with open("Catullus/TEXT-G2-pie.txt", "r") as input_text:
    print(input_text.read())

token	Case	Mood	Numb	Person	Tense	lemma	pos
fletus	Acc	_	Sing	_	_	fletus	NOMcom
passeris	Gen	_	Sing	_	_	passer	NOMcom
lesbie	Voc	_	Sing	_	_	lesbie	ADJqua
Passer	Voc	_	Sing	_	_	passer	NOMpro
delicie	_	Imp	Sing	2	Pres	delicio	VER
mee	Voc	_	Sing	_	_	ego	PROpos
puelle	Abl	_	Sing	_	_	puellis	NOMcom
Qui	Nom	_	Sing	_	_	quinus	NOMpro
cum	_	_	_	_	_	cum	PRE
ludere	_	Inf	_	_	Pres	ludo	VER
quem	Acc	_	Sing	_	_	qui	PROrel
in	_	_	_	_	_	in	PRE
sinu	Abl	_	Sing	_	_	sinus	NOMcom
tenere	_	Inf	_	_	Pres	teneo	VER
Qui	Nom	_	Sing	_	_	quinus	NOMpro
primum	_	_	_	_	_	primum	ADJadv.ord
digitum	Acc	_	Sing	_	_	digitus	NOMcom
dare	_	Inf	_	_	Pres	do	VER
at	_	_	_	_	_	at	CONcoo
patenti	Abl	_	Sing	_	Pres	patens	ADJqua
Et	_	_	_	_	_	et	ADV
acris	Acc	_	Plur	_	_	acer	ADJqua
solet	_	Ind	Sing	3	Pres	soleo	VER
incitare	_	Inf	_	_	Pres	incito	VER
morsus	Acc	_	Plur	_	_	morsus	NOMcom
Cum	Acc	_	Sing	_	_	cum	NOMpro
desiderio	Abl	_	Sing	_	_	desiderium	NOMcom
meo	Abl	_	Sing	_	_	meus	PROpos
nitenti	Dat	Par	Sing	_	Pres	nitor	VER
Carum	A

Cela semble avoir marché. Passons à l'étape suivante. Nous voulons pouvoir inclure les annotations linguistiques dans le processus de collation. Pour ce faire, il faut utiliser un format spécifique qu'est le JSON. Nous allons voir comment faire.

In [97]:

# On va importer les données lemmatisées
with open("Catullus/TEXT-G2-pie.txt", "r") as input_text:
    text_1 = [line.replace("\n", "") for line in input_text.readlines()][1:]

# et 2
with open("Catullus/TEXT-O1-pie.txt", "r") as input_text:
    text_2 = [line.replace("\n", "") for line in input_text.readlines()][1:]


# et 3
with open("Catullus/TEXT-Bodmer47-1-pie.txt", "r") as input_text:
    text_3 = [line.replace("\n", "") for line in input_text.readlines()][1:]

In [138]:
def import_annotated_data(input_text:str):
    output_list = []
    for line in input_text:
        splits = line.split("\t")
        print(splits)
        output_list.append(splits)
    return output_list

def create_json_input_for_collatex(dict_of_text, collate_on="lemmas"):
    output_dict = {"witnesses": []}
    for sigla, text in dict_of_text.items():
        witness_dict = {"id": sigla}
        tokens = []
        for token in text:
            form, case, mood, numb, person, tense, lemma, pos = token
            morph = case+mood+numb+person+tense
            if collate_on == "lemmas":
                tokens.append({"t": form, "n":lemma, "lemma": lemma, "pos": pos, "morph": morph})
            elif collate_on == "lemmas+pos":
                tokens.append({"t": form, "n":f"{lemma}|{pos}", "lemma": lemma, "pos": pos, "morph": morph})
            else:
                tokens.append({"t": form, "n":form, "lemma": lemma, "pos": pos, "morph": morph})
        witness_dict["tokens"] = tokens
        output_dict["witnesses"].append(witness_dict)
    return output_dict

In [139]:
text_1_as_list = import_annotated_data(text_1)
text_2_as_list = import_annotated_data(text_2)
text_3_as_list = import_annotated_data(text_3)
dict_of_text = {"O1": text_1_as_list, "G2": text_2_as_list, "Bodmer47": text_3_as_list}


json_input_forms = create_json_input_for_collatex(dict_of_text, collate_on="forms")

['fletus', 'Acc', '_', 'Sing', '_', '_', 'fletus', 'NOMcom']
['passeris', 'Gen', '_', 'Sing', '_', '_', 'passer', 'NOMcom']
['lesbie', 'Voc', '_', 'Sing', '_', '_', 'lesbie', 'ADJqua']
['Passer', 'Voc', '_', 'Sing', '_', '_', 'passer', 'NOMpro']
['delicie', '_', 'Imp', 'Sing', '2', 'Pres', 'delicio', 'VER']
['mee', 'Voc', '_', 'Sing', '_', '_', 'ego', 'PROpos']
['puelle', 'Abl', '_', 'Sing', '_', '_', 'puellis', 'NOMcom']
['Qui', 'Nom', '_', 'Sing', '_', '_', 'quinus', 'NOMpro']
['cum', '_', '_', '_', '_', '_', 'cum', 'PRE']
['ludere', '_', 'Inf', '_', '_', 'Pres', 'ludo', 'VER']
['quem', 'Acc', '_', 'Sing', '_', '_', 'qui', 'PROrel']
['in', '_', '_', '_', '_', '_', 'in', 'PRE']
['sinu', 'Abl', '_', 'Sing', '_', '_', 'sinus', 'NOMcom']
['tenere', '_', 'Inf', '_', '_', 'Pres', 'teneo', 'VER']
['Qui', 'Nom', '_', 'Sing', '_', '_', 'quinus', 'NOMpro']
['primum', '_', '_', '_', '_', '_', 'primum', 'ADJadv.ord']
['digitum', 'Acc', '_', 'Sing', '_', '_', 'digitus', 'NOMcom']
['dare', '_', 'I

In [124]:
result_table_forms = collatex.collate(json_input_forms, output="html2", segmentation=False, near_match=True)

La table d'alignement sur les formes est de qualité moyenne, on y compte un certain nombre d'erreurs. Comment améliorer l'alignement ? On peut penser à améliorer la *normalisation* des données, en supprimant l'information graphique et grammaticale: c'est ce que fait la **lemmatisation**. Alignons donc sur les lemmes:

In [120]:

json_input_lemmas = create_json_input_for_collatex(dict_of_text, collate_on="lemmas")
result_table_lemmas = collatex.collate(json_input_lemmas, output="html2", segmentation=False, near_match=True)
result_table_lemmas_pos = collatex.collate(json_input_lemmas_pos, output="html2", segmentation=False, near_match=True)
json_input_lemmas_pos = create_json_input_for_collatex(dict_of_text, collate_on="lemmas+pos")
print(result_table_lemmas)

None


Le résultat est meilleur: le début du texte est aligné de façon correcte, mais il reste quelques erreurs (`digitum dare at petenti` en est un bon exemple). 

In [119]:
json_input_lemmas_pos = create_json_input_for_collatex(dict_of_text, collate_on="lemmas+pos")
result_table_lemmas_pos = collatex.collate(json_input_lemmas_pos, output="html2", segmentation=False, near_match=True)
print(result_table_lemmas_pos)

None


In [130]:
resultat_json = collatex.collate(json_input_lemmas_pos, output='json', segmentation=False, near_match=True)
alignment_results_as_json = json.loads(resultat_json)


In [136]:
# Cette cellule charge les fonctions principales permettant d'analyser les variantes

def check_morph(locus):
    all_morphs = [witness['morph'] for witness in locus]
    print(f"Vérifions la morphologie: {all_morphs}")
    if all([morph == all_morphs[0] for morph in all_morphs[1:]]):
        print("La morphologie est identique.")
        return {'morph': True}
    else:
        print("Une différence morphologique semble apparaître: variante syntaxique ou grammaticale")
        return {'morph': False}

def check_annotations(locus):
    all_lemmas = [witness['lemme'] for witness in locus]
    print(all_lemmas)
    all_lemmas_as_string = " | ".join(all_lemmas)
    print(f"Vérifions les lemmes: {all_lemmas_as_string}")
    if all([lemma == all_lemmas[0] for lemma in all_lemmas[1:]]):
        print("Les lemmes sont identiques.")
        return {**check_morph(locus), **{"lemmas": True}}
    else:
        print("Les lemmes sont distincts. Variante lexicale")
        return {"lemmas": False, "morph":"UNK"}

def simplify_results(alignment_results_as_json):
    zipped = list(zip(alignment_results_as_json['table'][0], alignment_results_as_json['table'][1], alignment_results_as_json['table'][2]))

    output_data = list()
    for locus in zipped:
        interm_list = []
        for index, witness in enumerate(locus):
            if witness is not None:
                interm_dict = dict()
                interm_dict['témoin'] = witness[0]['_sigil']
                interm_dict['forme'] = witness[0]['t']
                interm_dict['lemme'] = witness[0]['n']
                interm_dict['pos'] = witness[0]['pos']
                interm_dict['morph'] = witness[0]['morph']
                interm_list.append(interm_dict)
            else:
                interm_dict = dict()
                interm_dict['témoin'] = alignment_results_as_json["witnesses"][index]
                interm_dict['forme'] = None
                interm_dict['lemme'] = None
                interm_dict['morph'] = None
                interm_dict['pos'] = None
                interm_list.append(interm_dict)
        output_data.append(interm_list)
    return output_data

def analyse_lieux_variants(collatex_output):
    results = simplify_results(collatex_output)
    # On crée une boucle sur chaque mot aligné
    for locus in results:
        print(f"Nouveau lieu variant.")
        # On commence par comparer les formes
        print(f"Comparons les formes: {' | '.join([witness['forme'] if witness['forme'] != None else 'ø' for witness in locus])}")
        forme_base = locus[0]['forme']
        print(f"La forme base de la comparaison est: {forme_base}")
        # Si toutes les formes sont identiques entre elles, alors il n'y a pas de lieu variant.
        if all([witness['forme'] == forme_base for witness in locus]):
            print("Toutes les formes sont identiques, il n'y a pas de lieu variant.")
            
        # Au contraire, s'il y a une divergence formelle, il faut creuser pour voir si il s'agit d'une variante

        # Un cas possible est celui de l'omission d'un des témoins
        elif any([witness['forme'] == None for witness in locus]):
            all_forms = [witness['forme'] for witness in locus if witness['forme'] != None]
            all_forms_as_string = " | ".join(all_forms)
            print(f"On note une omission à cet endroit du texte. \nVérifions si les autres témoins concordent: {all_forms_as_string}")
            
            # Si les autres témoins concordent, il s'agit d'un lieu variant avec omission d'un témoin (ou plus) uniquement
            if all([form == all_forms[0] for form in all_forms[1:]]):
                print("Les autres témoins concordent. Omission")

            # Dans le cas inverse, il faut creuser pour voir s'il s'agit d'une variante
            else:
                print("Les autres témoins discordent dans leur forme")
                locus = [witness for witness in locus if witness['forme'] != None]
                # On va appeler une fonction qui vérifie d'abord si les lemmes concordent, puis si les parties du discours concordent.
                annotations_check = check_annotations(locus)

                # Si les lemmes et les parties du discours sont strictement identiques, nous avons une variante graphique
                if annotations_check['morph'] == True and annotations_check['lemmas'] == True:
                    print("Variante graphique")
        # Même processus que précédemment, mais sans omission.
        else:
            print("Les témoins discordent dans leur forme.")
            check_lemma = check_lemmas(locus)
            if check_lemma['morph'] == True and check_lemma['lemmas'] == True:
                print("Variante graphique")
            
    
        print("\n")

In [137]:
# On prend la sortie de collatex et on analyse chaque unité d'alignement à la suite
analyse_lieux_variants(alignment_results_as_json)

            

Nouveau lieu variant.
Comparons les formes: fletus | ø | Passeris
La forme base de la comparaison est: fletus
On note une omission à cet endroit du texte. 
Vérifions si les autres témoins concordent: fletus | Passeris
Les autres témoins discordent dans leur forme
['fletus|NOMcom', 'passeris|NOMpro']
Vérifions les lemmes: fletus|NOMcom | passeris|NOMpro
Les lemmes sont distincts. Variante lexicale


Nouveau lieu variant.
Comparons les formes: passeris | ø | appelatio
La forme base de la comparaison est: passeris
On note une omission à cet endroit du texte. 
Vérifions si les autres témoins concordent: passeris | appelatio
Les autres témoins discordent dans leur forme
['passer|NOMcom', 'appelatio|NOMcom']
Vérifions les lemmes: passer|NOMcom | appelatio|NOMcom
Les lemmes sont distincts. Variante lexicale


Nouveau lieu variant.
Comparons les formes: lesbie | ø | ø
La forme base de la comparaison est: lesbie
On note une omission à cet endroit du texte. 
Vérifions si les autres témoins conco

Comme on le voit, le processus est très sensible à la qualité de l'annotation et de la lemmatisation, qui est lui-même dépendant de la variabilité graphique des témoins. La phase d'annotation lexico-grammaticale est donc fondamentale et les modèles d'annotation doivent être le plus précis possible.